# How to set up workflows with Kili

In this tutorial, we'll learn how to set up basic Kili workflows:

1. [Managing reviews](#-Managing-reviews)
    1. [Placing a specific percentage of project assets in the review queue](#-Placing-a-specific-percentage-of-project-assets-in-the-review-queue)
    1. [Placing specific assets in the review queue](#-Placing-specific-assets-in-the-review-queue)
    1. [Sending an asset back to the labeling queue](#-Sending-an-asset-back-to-the-labeling-queue)
1. [Setting up consensus](#-Setting-up-consensus)
    1. [Setting consensus for a specific percentage of project assets](#-Setting-consensus-for-a-specific-percentage-of-project-assets)
    1. [Setting consensus for specific assets to compute consensus KPIs](#-Setting-consensus-for-specific-assets-to-compute-consensus-KPIs)
1. [Setting up honeypot](#-Setting-up-honeypot)
1. [Assigning labelers to assets](#-Assigning-labelers-to-assets)
1. [Prioritizing assets in the labeling queue](#-Prioritizing-assets-in-the-labeling-queue)

To work with this notebook, you'll have to install and instantiate Kili.

In [ ]:
!pip install --upgrade kili
from kili.client import Kili
import getpass

In [ ]:
KILI_API_KEY = getpass.getpass("Please enter your API key: ")

In [ ]:
kili = Kili(
    api_key=KILI_API_KEY,
    # api_endpoint="https://cloud.kili-technology.com/api/label/v2/graphql", 
    # the line above can be uncommented and changed if you are working with an on-premise version of Kili
)

In [ ]:
project_id = <TYPE_YOUR_PROJECT_ID_HERE>

For information on how to set up a Kili project, refer to the `basic_project_setup.ipynb` tutorial.

## Managing reviews

### Placing a specific percentage of project assets in the review queue 

You can set up the percentage of assets that will automatically appear in the review queue (1-100%).

In [327]:
def set_rev_queue_percentage(project_id: str, percentage: int) -> str:
    assert type(project_id) == str, "Project type must be a string!"
    assert type(percentage) == int, "Percentage must be an integer!"

    kili.update_properties_in_project(project_id=project_id, review_coverage=percentage)
    
    review_coverage = kili.projects(project_id=project_id, fields=['reviewCoverage'])
    print(f'The review coverage set up for project id {project_id} is {review_coverage[0]["reviewCoverage"]}%.')

set_rev_queue_percentage(project_id=project_id, percentage=50)

100%|██████████| 1/1 [00:00<00:00,  2.04it/s]

The review coverage set up for project id clbqdvzhy10i60lqb822o3ahr is 50%.


### Placing specific assets in the review queue

You can choose to set specific assets to be placed in the review queue.
First, let's simulate adding labels to some of our assets:

In [ ]:
def add_mock_labels_to_assets(project_id: str, asset_range: tuple) -> str:
    assert type(project_id) == str, "Project type must be a string!"
    assert type(asset_range) == tuple, "Asset range must be a tuple marking start and end asset numbers, for example: '(0, 40)'!"
    asset_id_array = []
    
    for asset_number in range(asset_range[0], asset_range[1]+1):
        try:
            asset_id = kili.assets(project_id=project_id)[asset_number]['id']
            asset_id_array.append(asset_id)
        except IndexError as e:
            available_assets = kili.count_assets(project_id=project_id)
            print(f"Asset number: {asset_number} does not exist in project id {project_id}. Available assets: 0 -> {available_assets-1}.")
            continue
    json_response_array = [{'JOB_0': {'categories': [{'confidence': 100, 'name': 'OBJECT_B'}]}} for i in range(asset_range[0], asset_range[1]+1)]

    kili.append_labels(asset_id_array=asset_id_array, json_response_array=json_response_array, label_type='DEFAULT')
    
    for asset_id in asset_id_array:
        current_asset_labels = kili.assets(project_id=project_id, asset_id=asset_id, fields=['labels.jsonResponse'])
        print(f"Current labels added to asset id: {asset_id}:\n{current_asset_labels}")

add_mock_labels_to_assets(project_id = project_id, asset_range = (0, 3))

Now, let's place some assets in the review queue:

In [326]:
def put_assets_in_rev_queue(project_id: str, asset_range: tuple) -> str:
    assert type(project_id) == str, "Project type must be a string!"
    assert type(asset_range) == tuple, "Asset range must be a tuple marking start and end asset numbers, for example: '(0, 40)'!"
    summary = []
    
    for asset_number in range(asset_range[0], asset_range[1]+1):
        try:
            asset_id = kili.assets(project_id=project_id)[asset_number]['id']
        except IndexError as e:
            available_assets = kili.count_assets(project_id=project_id)
            print(f"Asset number: {asset_number} does not exist in project id {project_id}. Available assets: 0 -> {available_assets-1}.")
            continue
        asset_status = kili.assets(project_id=project_id, asset_id=asset_id, fields=['status'])
        if asset_status[0]["status"] != "LABELED":
            print(f"Asset number: {asset_number}, asset id {asset_id}: current status is {asset_status[0]['status']}. Skipping.")
            continue

        kili.add_to_review([asset_id])

        asset_status = kili.assets(project_id=project_id, asset_id=asset_id, fields=['status'])
        summary.append((asset_number, asset_status[0]["status"]))
    for item in summary:
        print(f'The status for asset ID: {item[0]} is now {item[1]}.')

put_assets_in_rev_queue(project_id = project_id, asset_range = (0, 3))

100%|██████████| 1/1 [00:00<00:00,  2.05it/s]


Asset number: 0, asset id clbqdwqv300003b6gbepg5xop: current status is TO_REVIEW. Skipping.


100%|██████████| 1/1 [00:00<00:00,  2.04it/s]


Asset number: 1, asset id clbqdwqv300013b6gzpz47bat: current status is TO_REVIEW. Skipping.


100%|██████████| 1/1 [00:01<00:00,  1.41s/it]


Asset number: 2, asset id clbqdwqv300023b6gycknzs90: current status is TO_REVIEW. Skipping.


100%|██████████| 1/1 [00:00<00:00,  1.47it/s]

The status for asset ID: 3 is now TO_REVIEW.


For more information on asset statuses, refer to our [documentation](https://docs.kili-technology.com/docs/asset-lifecycle).

### Sending an asset back to the labeling queue

You can send specific labeled assets back to the labeling queue.
First, let's simulate adding labels to some of our assets:

In [325]:
def add_mock_labels_to_assets(project_id: str, asset_range: tuple) -> str:
    assert type(project_id) == str, "Project type must be a string!"
    assert type(asset_range) == tuple, "Asset range must be a tuple marking start and end asset numbers, for example: '(0, 40)'!"
    asset_id_array = []
    
    for asset_number in range(asset_range[0], asset_range[1]+1):
        try:
            asset_id = kili.assets(project_id=project_id)[asset_number]['id']
            asset_id_array.append(asset_id)
        except IndexError as e:
            available_assets = kili.count_assets(project_id=project_id)
            print(f"Asset number: {asset_number} does not exist in project id {project_id}. Available assets: 0 -> {available_assets-1}.")
            continue
    json_response_array = [{'JOB_0': {'categories': [{'confidence': 100, 'name': 'OBJECT_B'}]}} for i in range(asset_range[0], asset_range[1]+1)]

    kili.append_labels(asset_id_array=asset_id_array, json_response_array=json_response_array, label_type='DEFAULT')
    
    for asset_id in asset_id_array:
        current_asset_labels = kili.assets(project_id=project_id, asset_id=asset_id, fields=['labels.jsonResponse'])
        print(f"Current labels added to asset id: {asset_id}:\n{current_asset_labels}")

add_mock_labels_to_assets(project_id = project_id, asset_range = (0, 3))

100%|██████████| 1/1 [00:00<00:00,  2.04it/s]


Current labels added to asset id: clbqdwqv300003b6gbepg5xop:
[{'labels': [{'jsonResponse': {'JOB_0': {'categories': [{'confidence': 100, 'name': 'OBJECT_B'}]}}}]}]


100%|██████████| 1/1 [00:00<00:00,  2.05it/s]


Current labels added to asset id: clbqdwqv300013b6gzpz47bat:
[{'labels': [{'jsonResponse': {'JOB_0': {'categories': [{'confidence': 100, 'name': 'OBJECT_B'}]}}}]}]


100%|██████████| 1/1 [00:00<00:00,  2.03it/s]


Current labels added to asset id: clbqdwqv300023b6gycknzs90:
[{'labels': [{'jsonResponse': {'JOB_0': {'categories': [{'confidence': 100, 'name': 'OBJECT_B'}]}}}]}]


100%|██████████| 1/1 [00:00<00:00,  2.04it/s]

Current labels added to asset id: clbqdwqv300033b6gien5hmlt:
[{'labels': [{'jsonResponse': {'JOB_0': {'categories': [{'confidence': 100, 'name': 'OBJECT_B'}]}}}]}]


Now, we'll send some of our assets back to the labeling queue. The new status for the asset should be "ONGOING". 

In [324]:
def send_assets_back_to_queue(project_id: str, asset_range: tuple) -> str:
    assert type(project_id) == str, "Project type must be a string!"
    assert type(asset_range) == tuple, "Asset range must be a tuple marking start and end asset numbers, for example: '(0, 40)'!"
    asset_id_array = []
    summary = []

    for asset_number in range(asset_range[0], asset_range[1]+1):
        try:
            asset_id = kili.assets(project_id=project_id)[asset_number]['id']
            asset_id_array.append(asset_id)
        except IndexError as e:
            available_assets = kili.count_assets(project_id=project_id)
            print(f"Asset number: {asset_number} does not exist in project id {project_id}. Available assets: 0 -> {available_assets-1}.")
            continue

        kili.send_back_to_queue(asset_ids=asset_id_array)

    asset_info = zip(range(asset_range[0], asset_range[1]+1), asset_id_array)
    asset_info_list = list(asset_info)
    
    for asset_item in asset_info_list:
        asset_status = kili.assets(project_id=project_id, asset_id=asset_item[1], fields=['status'])
        summary.append((asset_item[0], asset_item[1], asset_status[0]["status"]))
    for item in summary:
        print(f'The status for asset number: {item[0]}, asset ID: {item[1]} is now {item[2]}.')

send_assets_back_to_queue(project_id = project_id, asset_range = (0, 3))

100%|██████████| 1/1 [00:00<00:00,  2.05it/s]

The status for asset number: 0, asset ID: clbqdwqv300003b6gbepg5xop is now ONGOING.
The status for asset number: 1, asset ID: clbqdwqv300013b6gzpz47bat is now ONGOING.
The status for asset number: 2, asset ID: clbqdwqv300023b6gycknzs90 is now ONGOING.
The status for asset number: 3, asset ID: clbqdwqv300033b6gien5hmlt is now ONGOING.


For more information on asset statuses, refer to our [documentation](https://docs.kili-technology.com/docs/asset-lifecycle).

## Setting up consensus

Consensus works by having more than one labeler annotate the same asset. When the asset is labeled, a consensus score is calculated to measure the agreement level between the different annotations for a given asset.
This is a key measure for controlling label production quality.

To set up consensus, you'll need to have at least two project members.
For information on how to add users and assign them to your project, refer to the `basic_project_setup.ipynb` tutorial.

In [ ]:
kili.count_project_users(project_id=project_id)

### Setting consensus for a specific percentage of project assets

In [323]:
def set_consensus_percentage(project_id: str, percentage: int) -> str:
    assert type(project_id) == str, "Project type must be a string!"
    assert type(percentage) == int, "Percentage must be an integer!"

    kili.update_properties_in_project(project_id=project_id, consensus_tot_coverage=percentage)
    
    cons_tot_coverage = kili.projects(project_id=project_id, fields=["consensusTotCoverage"])
    print(f'The consensus coverage for project ID {project_id} is now {cons_tot_coverage[0]["consensusTotCoverage"]}%.')

set_consensus_percentage(project_id=project_id, percentage=100)

100%|██████████| 1/1 [00:00<00:00,  2.03it/s]

The consensus coverage for project ID clbqdvzhy10i60lqb822o3ahr is now 100%.


### Setting consensus for specific assets to compute consensus KPIs

You can manually select specific project assets to be used for computing consensus KPIs:

In [322]:
def set_consensus_for_assets(project_id: str, asset_consensus_dict: dict) -> str:

    assert type(project_id) == str, "Project type must be a string!"
    assert type(asset_consensus_dict) == dict, "'asset_consensus_dict' must be a dict. Dict keys are are tuples marking start and end asset numbers, for example: '(0, 40)' and values are consensus values (True or False)."

    summary = []
    mutation_dict = {}
    list_of_updated_assets = []
    all_project_assets = kili.assets(project_id=project_id)

    for asset_list in asset_consensus_dict.keys():
        assert type(asset_list) == tuple, "Asset range must be a tuple marking start and end asset numbers, for example: '(0, 40)'!"
        consensus = asset_consensus_dict[asset_list]
        assert type(consensus) == bool, "Asset consensus value must be a True or False!"

        all_project_external_ids = {}
        for i in range(len(all_project_assets)):
            all_project_external_ids[all_project_assets[i]["externalId"]] = i
        for asset_name in asset_list:
            if asset_name not in all_project_external_ids.keys():
                print(f"Asset name {asset_name} not found in project. Skipping...")
                continue
            else:
                asset_id = all_project_assets[all_project_external_ids[asset_name]]['id']
                list_of_updated_assets.append((asset_id, asset_name))
            if asset_id not in mutation_dict.keys():
                mutation_dict[asset_id] = consensus
            else:
                print(f"Warning! Two consensus values set for one asset: {asset_id}. Using the latter.")
                mutation_dict[asset_id] = consensus

    asset_ids = [i for i in mutation_dict.keys()]
    is_used_for_consensus_array = [mutation_dict[key] for key in mutation_dict.keys()]
    
    kili.update_properties_in_assets(asset_ids=asset_ids, is_used_for_consensus_array=is_used_for_consensus_array)

    for asset in list_of_updated_assets:
        is_used_for_consensus = kili.assets(project_id=project_id, asset_id=asset[0], fields=["isUsedForConsensus"])
        summary.append((asset[0], asset[1], is_used_for_consensus[0]["isUsedForConsensus"]))
    for item in summary:
        print(f'AssetID: {item[0]}, asset name: {item[1]} is used for consensus: {item[2]}.')

asset_consensus_dict = {
    ("2.jpg", "3.jpg"): True,
    ("4.jpg", "5.jpg"): False
}

set_consensus_for_assets(project_id = project_id, asset_consensus_dict = asset_consensus_dict)

100%|██████████| 1/1 [00:00<00:00,  2.03it/s]

AssetID: clbqdwqv300013b6gzpz47bat, asset name: 2.jpg is used for consensus: True.
AssetID: clbqdwqv300023b6gycknzs90, asset name: 3.jpg is used for consensus: True.
AssetID: clbqdwqv300033b6gien5hmlt, asset name: 4.jpg is used for consensus: False.
AssetID: clbqdwqv300043b6gbifi2ovx, asset name: 5.jpg is used for consensus: False.


For more information on consensus, refer to our [documentation](https://docs.kili-technology.com/docs/consensus-overview).

## Setting up honeypot

Honeypot (or __gold standard__) is a tool for auditing the work of labelers by measuring the accuracy of their annotations.
Honeypot works by interspersing assets with defined ground truth label in the annotation queue. This way you can measure the agreement level between your ground truth and the annotations made by labelers.

You can manually select specific project assets to be used as honeypots:

In [321]:
def set_honeypot_for_assets(project_id: str, asset_honeypot_dict: dict) -> str:
    assert type(project_id) == str, "Project type must be a string!"
    assert type(asset_honeypot_dict) == dict, "'asset_honeypot_dict' must be a dict. Dict keys are are tuples marking start and end asset numbers, for example: '(0, 40)' and values are honeypot values (True or False)."

    summary = []
    mutation_dict = {}
    list_of_updated_assets = []
    all_project_assets = kili.assets(project_id=project_id)

    for asset_list in asset_honeypot_dict.keys():
        assert type(asset_list) == tuple, "Asset range must be a tuple containing asset names, for example: '0001.jpg, 0002.jpg'!"
        honeypot = asset_honeypot_dict[asset_list]
        assert type(honeypot) == bool, "Asset honeypot value must be a True or False!"

        all_project_external_ids = {}
        for i in range(len(all_project_assets)):
            all_project_external_ids[all_project_assets[i]["externalId"]] = i
        for asset_name in asset_list:
            if asset_name not in all_project_external_ids.keys():
                print(f"Asset name {asset_name} not found in project. Skipping...")
                continue
            else:
                asset_id = all_project_assets[all_project_external_ids[asset_name]]['id']
                list_of_updated_assets.append((asset_id, asset_name))
            if asset_id not in mutation_dict.keys():
                mutation_dict[asset_id] = honeypot
            else:
                print(f"Warning! Two honeypot values set for one asset: {asset_id}. Using the latter.")
                mutation_dict[asset_id] = honeypot

    asset_ids = [i for i in mutation_dict.keys()]
    is_honeypot_array = [mutation_dict[key] for key in mutation_dict.keys()]

    kili.update_properties_in_assets(asset_ids=asset_ids, is_honeypot_array=is_honeypot_array)

    for asset in list_of_updated_assets:
        is_used_for_hp = kili.assets(project_id=project_id, asset_id=asset[0], fields=["isHoneypot"])
        summary.append((asset[0], asset[1], is_used_for_hp[0]["isHoneypot"]))
    for item in summary:
        print(f'AssetID: {item[0]}, asset name: {item[1]} is used as honeypot: {item[2]}.')


asset_honeypot_dict = {
    ("2.jpg", "3.jpg"): True,
    ("4.jpg", "5.jpg"): False
}

set_honeypot_for_assets(project_id = project_id, asset_honeypot_dict = asset_honeypot_dict)

100%|██████████| 1/1 [00:00<00:00,  1.95it/s]

AssetID: clbqdwqv300013b6gzpz47bat, asset name: 2.jpg is used as honeypot: 2.jpg.
AssetID: clbqdwqv300023b6gycknzs90, asset name: 3.jpg is used as honeypot: 3.jpg.
AssetID: clbqdwqv300033b6gien5hmlt, asset name: 4.jpg is used as honeypot: 4.jpg.
AssetID: clbqdwqv300043b6gbifi2ovx, asset name: 5.jpg is used as honeypot: 5.jpg.


For more information on honeypot, refer to our [documentation](https://docs.kili-technology.com/docs/consensus-overview).

## Assigning labelers to assets

You can assign specific labelers to specific assets in your project. You can do tht by assigning users' emails to the selected asset IDs:

In [320]:
def assign_labeler_to_assets(project_id: str, asset_email_dict: dict) -> str:

    assert type(project_id) == str, "Project type must be a string!"
    assert type(asset_email_dict) == dict, "'asset_email_dict' must be a dict. Dict keys are are tuples marking start and end asset numbers, for example: '(0, 40)' and values are user e-mails."

    summary = []
    mutation_dict = {}
    list_of_updated_assets = []
    all_project_assets = kili.assets(project_id=project_id)

    for asset_list in asset_email_dict.keys():
        assert type(asset_list) == tuple, "Asset range must be a tuple containing asset names, for example: '0001.jpg, 0002.jpg'!"
        email = asset_email_dict[asset_list]
        assert type(email) == str, "User email must be a string!"
        assert "@" in email, f"'@' missing in email address {email}. Check email address!"
        
        all_project_external_ids = {}
        for i in range(len(all_project_assets)):
            all_project_external_ids[all_project_assets[i]["externalId"]] = i
        for asset_name in asset_list:
            if asset_name not in all_project_external_ids.keys():
                print(f"Asset name {asset_name} not found in project. Skipping...")
                continue
            else:
                asset_id = all_project_assets[all_project_external_ids[asset_name]]['id']
                list_of_updated_assets.append((asset_id, asset_name))
                if asset_id not in mutation_dict.keys():
                    mutation_dict[asset_id] = [email]
                else:
                    mutation_dict[asset_id].append(email)

    asset_ids = [i for i in mutation_dict.keys()]
    to_be_labeled_by_array = [mutation_dict[key] for key in mutation_dict.keys()]

    kili.update_properties_in_assets(asset_ids=asset_ids, to_be_labeled_by_array=to_be_labeled_by_array)

    for asset in list_of_updated_assets:
        to_be_labeled_by = kili.assets(project_id=project_id, asset_id=asset[0], fields=["toBeLabeledBy.user.email"])
        list_of_labelers = to_be_labeled_by[0]["toBeLabeledBy"]
        to_be_labeled_by = list_of_labelers[0]["user"]["email"]
        if len(list_of_labelers) > 1:
            for i in range(1, len(list_of_labelers)):
                to_be_labeled_by += f", {list_of_labelers[i]['user']['email']}"
        summary.append((asset[0], asset[1], to_be_labeled_by))
    for item in summary:
        print(f'AssetID: {item[0]}, asset name: {item[1]} is set to be labeled by: {item[2]}.')

asset_email_dict = {
    ("2.jpg", "3.jpg"): "some.email@kili-technology.com",
    ("4.jpg", "5.jpg"): "some.email@kili-technology.com"
}

assign_labeler_to_assets(project_id = project_id, asset_email_dict = asset_email_dict)

100%|██████████| 1/1 [00:00<00:00,  2.04it/s]

AssetID: clbqdwqv300013b6gzpz47bat, asset name: 2.jpg is set to be labeled by: przemyslaw.czuba@kili-technology.com.
AssetID: clbqdwqv300023b6gycknzs90, asset name: 3.jpg is set to be labeled by: przemyslaw.czuba@kili-technology.com.
AssetID: clbqdwqv300033b6gien5hmlt, asset name: 4.jpg is set to be labeled by: przemyslaw.czuba@kili-technology.com.
AssetID: clbqdwqv300043b6gbifi2ovx, asset name: 5.jpg is set to be labeled by: przemyslaw.czuba@kili-technology.com.


For information on how to add users and assign them to your project, refer to the `basic_project_setup.ipynb` tutorial.
For information on assigning assets to users, refer to our [documentation](https://docs.kili-technology.com/docs/queue-prioritization).

## Prioritizing assets in the labeling queue

If you have certain assets that you need to have labeled earlier than the rest, you can use Kili's asset prioritization methods.

In [319]:
def set_priorities_for_assets(project_id: str, asset_priority_dict: dict) -> str:

    assert type(project_id) == str, "Project type must be a string!"
    assert type(asset_priority_dict) == dict, "'asset_priority_dict' must be a dict. Dict keys are lists containing asset names, for example: '0001.jpg' and values are specific priorities."

    summary = []
    mutation_dict = {}
    list_of_updated_assets = []
    all_project_assets = kili.assets(project_id=project_id)

    for asset_list in asset_priority_dict.keys():
        assert type(asset_list) == tuple, "Asset range must be a tuple containing asset names, for example: '0001.jpg, 0002.jpg'!"
        priority = asset_priority_dict[asset_list]
        assert type(priority) == int, "Asser priority must be an integer!"
        
        all_project_external_ids = {}
        for i in range(len(all_project_assets)):
            all_project_external_ids[all_project_assets[i]["externalId"]] = i
        for asset_name in asset_list:
            if asset_name not in all_project_external_ids.keys():
                print(f"Asset name {asset_name} not found in project. Skipping...")
                continue
            else:
                asset_id = all_project_assets[all_project_external_ids[asset_name]]['id']
                list_of_updated_assets.append((asset_id, asset_name))
            if asset_id not in mutation_dict.keys():
                mutation_dict[asset_id] = priority
            else:
                print(f"Warning! Two priorities set for one asset: {asset_id}. Using the latter.")
                mutation_dict[asset_id] = priority

    asset_ids = [i for i in mutation_dict.keys()]
    priorities = [mutation_dict[key] for key in mutation_dict.keys()]

    kili.update_properties_in_assets(asset_ids=asset_ids, priorities=priorities)

    for asset in list_of_updated_assets:
        priority = kili.assets(project_id=project_id, asset_id=asset[0], fields=["priority"])
        summary.append((asset[0], asset[1], priority[0]["priority"]))
    for item in summary:
        print(f'Asset ID: {item[0]}, asset name: {item[1]} is set to priority: {item[2]}.')

asset_priority_dict = {
    ("1.jpg", "2.jpg"): 0,
    ("3.jpg", "4.jpg"): 2
}

set_priorities_for_assets(project_id = project_id, asset_priority_dict = asset_priority_dict)

100%|██████████| 1/1 [00:00<00:00,  2.04it/s]

Asset ID: clbqdwqv300003b6gbepg5xop, asset name: 1.jpg is set to priority: 0.
Asset ID: clbqdwqv300013b6gzpz47bat, asset name: 2.jpg is set to priority: 0.
Asset ID: clbqdwqv300023b6gycknzs90, asset name: 3.jpg is set to priority: 2.
Asset ID: clbqdwqv300033b6gien5hmlt, asset name: 4.jpg is set to priority: 2.


For information on setting asset priorities, refer to our [documentation](https://docs.kili-technology.com/docs/queue-prioritization).

## Summary

Done. We've learned how to handle the review workflow, set up consensus and honeypot in a project, assign specific labelers to specific assets, and how to prioritize assets in the labeling queue. Well done!